# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [ ]:
# !pip install scikit-surprise

간단한 surprise 실습

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [4]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [5]:
model = SVD()

In [6]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9329  0.9389  0.9372  0.9297  0.9391  0.9355  0.0037  
MAE (testset)     0.7351  0.7421  0.7391  0.7337  0.7402  0.7380  0.0032  
Fit time          0.72    0.68    0.70    0.70    0.72    0.70    0.01    
Test time         0.10    0.07    0.11    0.07    0.07    0.08    0.02    


{'test_rmse': array([0.93287734, 0.93889655, 0.93716155, 0.92965889, 0.93913615]),
 'test_mae': array([0.73509109, 0.74209276, 0.73906204, 0.7337023 , 0.74023386]),
 'fit_time': (0.7159991264343262,
  0.6755461692810059,
  0.7021200656890869,
  0.6977996826171875,
  0.7150166034698486),
 'test_time': (0.10100173950195312,
  0.06907081604003906,
  0.10699796676635742,
  0.07397103309631348,
  0.06799888610839844)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [7]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [8]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [9]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [10]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [15]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [17]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))
    

Best Match: 183, Best Match ID: 275


In [18]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [19]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 14.832396974191326, Best Match ID: 737


In [20]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [22]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)


In [24]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 0.5278586163659506, Best Match ID: 915


In [25]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [26]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [27]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 55.06359959174482, Best Match ID: 737


In [28]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 0.569065731527988, Best Match ID: 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [29]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [30]:
data = Dataset.load_builtin('ml-100k', prompt=False)

* KNN을 사용한 협업 필터링

In [31]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9751  0.9724  0.9809  0.9847  0.9809  0.9788  0.0044  
MAE (testset)     0.7719  0.7680  0.7766  0.7755  0.7724  0.7729  0.0030  
Fit time          0.31    0.30    0.29    0.28    0.23    0.28    0.03    
Test time         1.56    1.59    1.51    1.54    1.35    1.51    0.09    


{'test_rmse': array([0.97514434, 0.97237306, 0.98087347, 0.98473856, 0.98086913]),
 'test_mae': array([0.77194409, 0.76799609, 0.77655081, 0.77550877, 0.77235122]),
 'fit_time': (0.30699992179870605,
  0.3039672374725342,
  0.29397010803222656,
  0.2799704074859619,
  0.23196649551391602),
 'test_time': (1.5608813762664795,
  1.5919065475463867,
  1.5119194984436035,
  1.538881778717041,
  1.3465759754180908)}

* SVD를 사용한 협업 필터링

In [32]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9371  0.9410  0.9316  0.9272  0.9419  0.9358  0.0056  
MAE (testset)     0.7363  0.7406  0.7368  0.7333  0.7441  0.7382  0.0037  
Fit time          0.49    0.49    0.51    0.49    0.45    0.49    0.02    
Test time         0.10    0.10    0.08    0.08    0.09    0.09    0.01    


{'test_rmse': array([0.93709172, 0.94101857, 0.93159042, 0.92722562, 0.94194129]),
 'test_mae': array([0.73634796, 0.74059378, 0.73682878, 0.73325648, 0.74406009]),
 'fit_time': (0.4929332733154297,
  0.4859459400177002,
  0.5139410495758057,
  0.49494385719299316,
  0.45294642448425293),
 'test_time': (0.0999917984008789,
  0.10199189186096191,
  0.08099222183227539,
  0.07899332046508789,
  0.08699488639831543)}

* NMF를 사용한 협업 필터링

In [33]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9731  0.9632  0.9679  0.9627  0.9589  0.9651  0.0049  
MAE (testset)     0.7660  0.7579  0.7619  0.7547  0.7537  0.7588  0.0046  
Fit time          0.68    0.70    0.66    0.71    0.57    0.66    0.05    
Test time         0.07    0.07    0.08    0.06    0.07    0.07    0.01    


{'test_rmse': array([0.97311145, 0.96320482, 0.96787379, 0.96266553, 0.95886875]),
 'test_mae': array([0.76602844, 0.75786694, 0.76189151, 0.75472222, 0.75371994]),
 'fit_time': (0.6839230060577393,
  0.6999232769012451,
  0.6559264659881592,
  0.7099230289459229,
  0.5679352283477783),
 'test_time': (0.06898999214172363,
  0.0739905834197998,
  0.07998919486999512,
  0.05899453163146973,
  0.0660252571105957)}

* SVD++를 사용한 협업 필터링

In [34]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9238  0.9161  0.9169  0.9182  0.9269  0.9204  0.0042  
MAE (testset)     0.7285  0.7163  0.7184  0.7200  0.7269  0.7220  0.0048  
Fit time          11.48   11.52   11.46   11.45   10.10   11.20   0.55    
Test time         2.09    2.00    1.94    2.15    1.74    1.98    0.14    


{'test_rmse': array([0.92375784, 0.91613057, 0.91693108, 0.91817978, 0.92692555]),
 'test_mae': array([0.72852734, 0.71625452, 0.71838675, 0.71997454, 0.72689997]),
 'fit_time': (11.483341932296753,
  11.522326231002808,
  11.46136736869812,
  11.450372695922852,
  10.095010042190552),
 'test_time': (2.0894851684570312,
  1.9954750537872314,
  1.941481590270996,
  2.1474556922912598,
  1.7363927364349365)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [35]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization 
from surprise import Dataset

In [66]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [67]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [68]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

In [69]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [70]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [71]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [72]:
np.matmul(np.matmul(U, S), V)

array([[ 3.91732680e+00,  1.47276650e+00,  7.98262091e-01, ...,
         6.24908144e-04,  1.41100857e-02,  1.36545869e-02],
       [ 1.85777193e+00,  3.96190807e-01,  5.05705276e-01, ...,
         5.38862763e-03,  1.77237362e-03,  5.26972982e-04],
       [ 8.94989211e-01,  1.71578285e-01,  2.51738388e-01, ...,
         2.92094743e-03,  5.39934286e-04, -1.25730788e-04],
       ...,
       [ 9.92051704e-01,  2.10814754e-01,  2.70363094e-01, ...,
         2.89019146e-03,  9.34219087e-04,  2.66614978e-04],
       [ 1.30425371e+00,  5.27669977e-01,  2.50080129e-01, ...,
        -4.20679221e-04,  5.30525822e-03,  5.28069946e-03],
       [ 2.82999374e+00,  9.70812149e-01,  6.15871536e-01, ...,
         2.02091364e-03,  8.67740702e-03,  8.03108041e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [73]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 0.9999942291059927, Best Match ID: 235


In [74]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [75]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 0.9999999949431925, Best Match ID: 1287


In [76]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [77]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [78]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

In [79]:
np.matmul(A, B)

array([[3.71107433e+00, 1.48461856e+00, 7.39541570e-01, ...,
        3.64501983e-03, 1.45513751e-02, 1.44116215e-02],
       [2.11729713e+00, 2.37145679e-01, 5.51637757e-01, ...,
        4.76290749e-03, 2.84605930e-05, 0.00000000e+00],
       [9.85325089e-01, 1.10360320e-01, 2.56715279e-01, ...,
        2.21651094e-03, 1.32446863e-05, 0.00000000e+00],
       ...,
       [1.04478344e+00, 1.17019891e-01, 2.72206478e-01, ...,
        2.35026384e-03, 1.40439223e-05, 0.00000000e+00],
       [1.45769331e+00, 5.42108391e-01, 2.99217251e-01, ...,
        1.61232500e-03, 5.15892655e-03, 5.10748255e-03],
       [2.44709957e+00, 9.41278705e-01, 4.95671746e-01, ...,
        2.56934867e-03, 9.08400301e-03, 8.99501717e-03]])

* 사용자 기반 추천

In [80]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 0.9999942291059927, Best Match ID: 235


In [81]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천

In [82]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match: {}, Best Match ID: {}".format(best_match, best_match_id))

Best Match: 0.9999999949431925, Best Match ID: 1287


In [83]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 